<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Dodanie-dat-filmw/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [11]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-22 17:51:33--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz.1’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==> SIZE spark-3.3.1-bin-hadoop2.tgz ... 274099817
==> PASV ... done.    ==> RETR spark-3.3.1-bin-hadoop2.tgz ... done.
Length: 274099817 (261M) (unauthoritative)

spark-3.3.1-bin-had 100%[===================>] 261.40M  22.5MB/s    in 12s     

2022-11-22 17:51:48 (21.0 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz.1’ saved [274099817]



In [12]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [13]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [14]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-22 17:51:53--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.73, 13.249.98.61, 13.249.98.35, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236745635 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.1’

name.basics.tsv.gz. 100%[===================>] 225.78M  86.0MB/s    in 2.6s    

2022-11-22 17:51:56 (86.0 MB/s) - ‘name.basics.tsv.gz.1’ saved [236745635/236745635]

--2022-11-22 17:51:56--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.73, 13.249.98.61, 13.249.98.35, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288061886 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.1’

title.akas.tsv.gz.1 100%[===================>] 274.72M 

##Rozpakowanie danych

In [15]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [16]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [17]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt00383...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00564...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy z tabel tylko istotne rekordy:
- df_title_basic: wybierzmy tylko rekordy z primaryProfession zawierającym "actor" lub "actress",
- df_title_principals: wybierzmy rekordy z category zawierającym "actor" lub "actress".

In [18]:
df_name_basics_selected = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_title_principals_selected = df_title_principals.filter((df_title_principals.category == "actor") | (df_title_principals.category == "actress"))

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - startYear (data wypuszczenia produkcji - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - endYear (data zakończenia serii, w innych przypadkach \N - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - knownForTitles (tytuły produkcji, z których znany jest aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [19]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "startYear", "endYear", "genres"])
df_title_principals_selected = df_title_principals_selected.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics_selected.select(["nconst", "primaryName", "knownForTitles"])

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [20]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (4427662, 3)
df_title_principals_selected dataframe size:  (20815083, 4)
df_title_basic_selected dataframe size:  (9388564, 7)
joined dataframe size:  (20815083, 10)
joined dataframe size:  (20190789, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [21]:
sample = data.rdd.takeSample(False, 5)

In [22]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,startYear,endYear,genres,category,characters,primaryName,knownForTitles
0,nm8057446,tt11506048,tvEpisode,Episode #1.60,0,2020,\N,"Drama,Romance",actress,"[""Jeong Joo-Ri""]",Ga-rim Han,"tt12929796,tt10808968,tt4846006,tt12035244"
1,nm0034498,tt4945096,tvEpisode,Tres vidas para una muerte,0,1964,\N,\N,actor,\N,Alberto Argibay,"tt0291619,tt0498422,tt2270143,tt5252240"
2,nm0293659,tt1212967,tvEpisode,Return of the Mole Man,0,1967,\N,"Action,Adventure,Animation",actor,"[""The Thing"",""Benjamin J. Grimm""]",Paul Frees,"tt0046534,tt0053640,tt0084237,tt0056545"
3,nm0694742,tt10154630,tvEpisode,El secuestro de Iván,0,2019,\N,Romance,actor,"[""Diego Jáuregui""]",Jorge Poza,"tt9104300,tt0296373,tt0266177,tt1195884"
4,nm0073512,tt5785994,tvEpisode,Drömmen om scoopet,0,1985,\N,Drama,actor,"[""Lennart""]",Lars-Erik Berenett,"tt0094493,tt0096717,tt2071471,tt0208198"
